## 1. Import Library

### 1.1. Import Standard Library

In [ ]:
import itertools
import pandas as pd
import math

### 1.2. Import Parser Library

In [ ]:
import requests
from bs4 import BeautifulSoup

### 1.3. Import Interactive Graphing Library: Bokeh

In [ ]:
from bokeh.palettes import viridis
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, HoverTool,  NumeralTickFormatter
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot

## 2. Web Scraping

### 2.1. Scraping Contents of Main Page

In [ ]:
resp = requests.get('http://www.stockq.org/market/commodity.php')
resp.encoding = 'utf-8'
soup = BeautifulSoup(resp.text, 'html.parser')
print(soup)

### 2.2. Scraping Contents of Subpages

In [ ]:
# Tag "a" is included in Tag "td."
td = soup.find_all('td')
# Tag "a" contains hyperlinks of listed commodities.
hrefs = []
comms = []

for s in td:
    try:
        a = s.find('a', href=True)
        hrefs.append("http://www.stockq.org" + a['href'])
        comms.append(s.find('a').text)
    except:
        pass

## 3. Clean Dataset

### 3.1. Extract Data

In [ ]:
# Ignore data that is not relevant to commodites.   
comms = comms[11:-2]
# Specify hyperlinks of corresponding commodities.
hrefs = hrefs[11:-2]

date = []
price = []

for href in hrefs: 
    
    # Extract data from the subpage of corresponding commoditiy. . 
    resp = requests.get(href)
    resp.encoding='utf-8'
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    # Price for each trading session of the specified commodity is listed in:
    # Level 1: 3rd Tag "table" with Class named "indexpagetable."
    # Level 2: Tag "tr" with Class named "row1" and "row2,"
    # where "row1" stands for "odd rows" of the table
    # while "row2" stands for "even rows" of the table.
    # Level 3: 1st and 4th Tag "td" include the "dates" of trading sessions
    # while 2nd and 5th Tag "td" include the corresponding prices.
    tables = soup.find_all('table', {'class': 'indexpagetable'})
    table = tables[2]

    rows = table.find_all('tr', {'class': ['row1', 'row2']})    

    # .find: find the 1st one.
    for row in rows:
        td = row.find('td')
        date.append(td.text)   

    for row in rows:
        td = row.find_all('td')[3]
        date.append(td.text)

    for row in rows:
        td = row.find_all('td')[1]
        price.append(td.text)

    for row in rows:
        td = row.find_all('td')[4]
        price.append(td.text)  
    
    # Make the length of "List of Commodities" identical to that of "date" and "price" data
    # by repeating each commodity the number of times of the occurance of "date" data.   
    n = int(len(date)/len(comms))
    comm = list(itertools.chain.from_iterable((itertools.repeat(comm, n) for comm in comms)))

### 3.2. Create Dataframe

In [ ]:
df = pd.DataFrame({'comm':comm, 'date': date, 'price': price})

### 3.3. Save Dataframe

In [ ]:
df.to_csv("G:\\Commodity_201812_201901.csv", index = False, encoding = "ansi")

### 3.4. Read csv File

In [ ]:
df = pd.read_csv("G:\\Commodity_201812_201901.csv", encoding = 'ansi')

### 3.5. Remove Data

In [ ]:
# Specify commodies that are not within the analysis period.  
df[df['date'].str.contains('2018/09')]

In [ ]:
# Exclude data that may not be correctly given.
df = df[~df['date'].str.contains('2019/12/31')]

In [ ]:
# Exclude the specified data from the dataframe.
df = df[df['comm'] != '鈾']
df

In [ ]:
# Remove the specified commodity from the list.
comms.remove('鈾')
comms

## 4. Rearrange Dataframe

### 4.1. Split Dataframe into Chunks

In [ ]:
# Create a list to include 26 small chunks which represent 26 listed commodities.
list_df = []

# "n" as key represents "comm."
# "g" as value includes "date" and "price."
for n,g in df.groupby('comm'):
    list_df.append(g)

### 4.2. Combine Chunks to Single Dataframe

In [ ]:
# Will be used to calculate the mean of each commodity.
df_mean = pd.concat(list_df)

### 4.3. Calculate Mean

In [ ]:
means= []
for i in range (len(list_df)):
    mean = sum((list_df)[i]['price'].astype(float))/len((list_df)[i])
    # Specify how many times each commodity has to repeat itself 
    # to make the length of the mean identical to that of “date” and “price.”
    n = len((list_df)[i])
    # Use "iteration" to perform repetitive calculations.
    m = list((itertools.repeat(mean, n)))
    # Add lists together.
    means = means + m

### 4.4. Add a New Column to Dataframe

In [ ]:
# Add a new column.
df_mean['mean'] = means
# Format values in Column “mean.”
df_mean['mean'] = df_mean['mean'].round(2)

### 4.5. Sort Values by Mean

In [ ]:
# Sort values in descending order.
df_mean = df_mean.sort_values(['mean', 'date'], ascending = False)

### 4.6. Groupby by Mean

In [ ]:
# Note that the sorting order of "groupby" function is defaulted as ascending, 
# which is why sorting values in descending order has to be done prior to the 
# application of "groupby" function and parameter "sort" must be set as "False." 
grouped = df_mean.groupby(["mean"], sort = False)

### 4.7. Extract Mean

In [ ]:
mean_keys = grouped.groups.keys()

### 4.8. Define the Number of Lines to be Displayed

In [ ]:
# Round a number upward to its nearest integer.
n = math.ceil(len(mean_keys)/4)

### 4.9. Split Means into Groups

In [ ]:
# Create a list to include 4 groups of means.
mean_data = []
for i in range (0, len(mean_keys), n):
    mean_data.append(list(mean_keys)[i:i+n]) 

### 4.10. Split Dataframe into Chunks

In [ ]:
# Create a list to include 4 lists each of which contains at most 6 dataframes.
mean_gp = []
# "i" used to specify which mean group is to be processed.
for i in range (len(mean_data)):
    l = [] 
    # "j" used to specify which element of the selected mean group is to be processed.
    for j in range (len((mean_data)[i])):
        temp = df_mean[df_mean['mean'] == mean_data[i][j]]
        l.append(temp)
    mean_gp.append(l)

## 5. Data Visualization

In [ ]:
# Configure an output to a standalone HTML file.
output_file('5_InteractiveLineChart.html')

In [ ]:
# Create a list to include 4 plots which will be arranged into a grid.
g = []

# Specify the display of tooltips.
# {%F} suggests that datetime will be formatted as YYYY-MM-DD.
# Notice: Tooltip data is associated with "source."
hover = HoverTool(tooltips=[('Commodity', '@Commodity'),('Date', '@Date{%F}'),('Price', '@Price')],
                  formatters={'Date': 'datetime'})

# Specify which mean group is to be processed.
for i in range(len(mean_gp)):

    # Set overall formats of the plot.
    p = figure(plot_width=800, plot_height=800, x_axis_type='datetime')
    p.title.text = 'Commodity Prices: Group%d'% (i+1)
    p.add_tools(hover)

    # Specify which element of the selected mean group is to be processed.
    for df, color in zip(mean_gp[i], viridis(7)):
        
        # Define legend items.
        name = df['comm'].unique()[0]

        # Convert argument to datetime.
        df['date'] = pd.to_datetime(df['date'])
        # Define data that is to be imported for plotting.
        source = ColumnDataSource(data={'Commodity': df['comm'], 'Date': df['date'], 'Price': df['price']})
        # Set overall formats of the line chart.
        # Notice: Data is associated with "source."
        r = p.line(x='Date', y='Price', line_width=2, color=color, alpha=0.8,
                   muted_color=color, muted_alpha=0.2, legend=name, source=source)
        # Default line chart as muted.
        r.muted = True

        # Location of legend.
        p.legend.location = 'top_left'
        # click on the legend item and the corresponding line will be muted or lighted up. 
        p.legend.click_policy = 'mute'
        # Add thousand separator to labels of Y-axis. 
        p.yaxis.formatter = NumeralTickFormatter(format='0,0')
        
    g.append(p)  
    
grid = gridplot(g, ncols=2, nrows=2, plot_width=400, plot_height=400)

output_notebook()
show(grid)